### MNISTサンプル

In [1]:
import caffe

パスを設定する。

In [2]:
import sys
import os
caffe_root = '/opt/caffe'
sys.path.insert(0, caffe_root + 'python')
os.chdir(caffe_root)
!data/mnist/get_mnist.sh
!examples/mnist/create_mnist.sh
os.chdir('examples')

Downloading...
Creating lmdb...
libdc1394 error: Failed to initialize libdc1394
I0817 23:45:39.200397   290 db_lmdb.cpp:35] Opened lmdb examples/mnist/mnist_train_lmdb
I0817 23:45:39.200837   290 convert_mnist_data.cpp:88] A total of 60000 items.
I0817 23:45:39.200861   290 convert_mnist_data.cpp:89] Rows: 28 Cols: 28
I0817 23:45:40.595028   290 convert_mnist_data.cpp:108] Processed 60000 files.
libdc1394 error: Failed to initialize libdc1394
I0817 23:45:40.646076   291 db_lmdb.cpp:35] Opened lmdb examples/mnist/mnist_test_lmdb
I0817 23:45:40.646564   291 convert_mnist_data.cpp:88] A total of 10000 items.
I0817 23:45:40.646591   291 convert_mnist_data.cpp:89] Rows: 28 Cols: 28
I0817 23:45:40.872876   291 convert_mnist_data.cpp:108] Processed 10000 files.
Done.


In [9]:
!ls -l mnist

total 3520
-rw-r--r-- 1 root root    4520 Aug 17 05:23 convert_mnist_data.cpp
-rwxr-xr-x 1 root root     634 Aug 17 05:23 create_mnist.sh
-rw-r--r-- 1 root root    1738 Aug 17 05:23 lenet.prototxt
-rw-r--r-- 1 root root     777 Aug 17 05:23 lenet_adadelta_solver.prototxt
-rw-r--r-- 1 root root     778 Aug 17 05:23 lenet_auto_solver.prototxt
-rw-r--r-- 1 root root    1407 Aug 17 09:14 lenet_auto_test.prototxt
-rw-r--r-- 1 root root    1407 Aug 17 09:14 lenet_auto_train.prototxt
-rw-r--r-- 1 root root    6003 Aug 17 05:23 lenet_consolidated_solver.prototxt
-rw-r--r-- 1 root root 1724885 Aug 17 09:27 lenet_iter_5000.caffemodel
-rw-r--r-- 1 root root 1724461 Aug 17 09:27 lenet_iter_5000.solverstate
-rw-r--r-- 1 root root     871 Aug 17 05:23 lenet_multistep_solver.prototxt
-rw-r--r-- 1 root root     790 Aug 17 05:23 lenet_solver.prototxt
-rw-r--r-- 1 root root     886 Aug 17 05:23 lenet_solver_adam.prototxt
-rw-r--r-- 1 root root     830 Aug 17 05:23 lenet_solver_rmsprop.prototxt
-rw-r--r-

In [10]:
train_net_path = 'mnist/custom_auto_train.prototxt'
test_net_path = 'mnist/custom_auto_test.prototxt'
solver_config_path = 'mnist/custom_auto_solver.prototxt'

In [11]:
caffe.set_mode_cpu()
solver = None
solver = caffe.SGDSolver('mnist/lenet_auto_solver.prototxt')

In [12]:
### define net
def custom_net(lmdb, batch_size):
    # define your own net!
    n = caffe.NetSpec()
    
    # keep this data layer for all networks
    n.data, n.label = L.Data(batch_size=batch_size, backend=P.Data.LMDB, source=lmdb,
                             transform_param=dict(scale=1./255), ntop=2)
    
    # EDIT HERE to try different networks
    # this single layer defines a simple linear classifier
    # (in particular this defines a multiway logistic regression)
    n.score =   L.InnerProduct(n.data, num_output=10, weight_filler=dict(type='xavier'))
    
    # EDIT HERE this is the LeNet variant we have already tried
    # n.conv1 = L.Convolution(n.data, kernel_size=5, num_output=20, weight_filler=dict(type='xavier'))
    # n.pool1 = L.Pooling(n.conv1, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    # n.conv2 = L.Convolution(n.pool1, kernel_size=5, num_output=50, weight_filler=dict(type='xavier'))
    # n.pool2 = L.Pooling(n.conv2, kernel_size=2, stride=2, pool=P.Pooling.MAX)
    # n.fc1 =   L.InnerProduct(n.pool2, num_output=500, weight_filler=dict(type='xavier'))
    # EDIT HERE consider L.ELU or L.Sigmoid for the nonlinearity
    # n.relu1 = L.ReLU(n.fc1, in_place=True)
    # n.score =   L.InnerProduct(n.fc1, num_output=10, weight_filler=dict(type='xavier'))
    
    # keep this loss layer for all networks
    n.loss =  L.SoftmaxWithLoss(n.score, n.label)
    
    return n.to_proto()

In [14]:
from caffe import layers as L, params as P
with open(train_net_path, 'w') as f:
    f.write(str(custom_net('mnist/mnist_train_lmdb', 64)))    
with open(test_net_path, 'w') as f:
    f.write(str(custom_net('mnist/mnist_test_lmdb', 100)))

In [19]:
### define solver
from pylab import *
from caffe.proto import caffe_pb2
s = caffe_pb2.SolverParameter()

# Set a seed for reproducible experiments:
# this controls for randomization in training.
s.random_seed = 0xCAFFE

# Specify locations of the train and (maybe) test networks.
s.train_net = train_net_path
s.test_net.append(test_net_path)
s.test_interval = 500  # Test after every 500 training iterations.
s.test_iter.append(100) # Test on 100 batches each time we test.

s.max_iter = 10000     # no. of times to update the net (training iterations)
 
# EDIT HERE to try different solvers
# solver types include "SGD", "Adam", and "Nesterov" among others.
s.type = "SGD"

# Set the initial learning rate for SGD.
s.base_lr = 0.01  # EDIT HERE to try different learning rates
# Set momentum to accelerate learning by
# taking weighted average of current and previous updates.
s.momentum = 0.9
# Set weight decay to regularize and prevent overfitting
s.weight_decay = 5e-4

# Set `lr_policy` to define how the learning rate changes during training.
# This is the same policy as our default LeNet.
s.lr_policy = 'inv'
s.gamma = 0.0001
s.power = 0.75
# EDIT HERE to try the fixed rate (and compare with adaptive solvers)
# `fixed` is the simplest policy that keeps the learning rate constant.
# s.lr_policy = 'fixed'

# Display the current training loss and accuracy every 1000 iterations.
s.display = 1000

# Snapshots are files used to store networks we've trained.
# We'll snapshot every 5K iterations -- twice during training.
s.snapshot = 5000
s.snapshot_prefix = 'mnist/custom_net'

# Train on the GPU
s.solver_mode = caffe_pb2.SolverParameter.CPU

# Write the solver to a temporary file and return its filename.
with open(solver_config_path, 'w') as f:
    f.write(str(s))

### load the solver and create train and test nets
solver = None  # ignore this workaround for lmdb data (can't instantiate two solvers on the same data)
solver = caffe.get_solver(solver_config_path)
solver.solve()
### solve
#niter = 250  # EDIT HERE increase to train for longer
#test_interval = niter / 10
# losses will also be stored in the log
#train_loss = zeros(niter)
#test_acc = zeros(int(np.ceil(niter / test_interval)))

# the main solver loop
#for it in range(niter):
#    solver.step(1)  # SGD by Caffe
    
    # store the train loss
#    train_loss[it] = solver.net.blobs['loss'].data
    
    # run a full test every so often
    # (Caffe can also do this for us and write to a log, but we show here
    #  how to do it directly in Python, where more complicated things are easier.)
#    if it % test_interval == 0:
#        print 'Iteration', it, 'testing...'
#        correct = 0
#        for test_it in range(100):
#            solver.test_nets[0].forward()
#            correct += sum(solver.test_nets[0].blobs['score'].data.argmax(1)
#                           == solver.test_nets[0].blobs['label'].data)
#        test_acc[it // test_interval] = correct / 1e4

In [21]:
!ls -l mnist

total 3660
-rw-r--r-- 1 root root    4520 Aug 17 05:23 convert_mnist_data.cpp
-rwxr-xr-x 1 root root     634 Aug 17 05:23 create_mnist.sh
-rw-r--r-- 1 root root     346 Aug 17 10:04 custom_auto_solver.prototxt
-rw-r--r-- 1 root root     490 Aug 17 09:58 custom_auto_test.prototxt
-rw-r--r-- 1 root root     490 Aug 17 09:58 custom_auto_train.prototxt
-rw-r--r-- 1 root root   31604 Aug 17 10:04 custom_net_iter_10000.caffemodel
-rw-r--r-- 1 root root   31469 Aug 17 10:04 custom_net_iter_10000.solverstate
-rw-r--r-- 1 root root   31604 Aug 17 10:04 custom_net_iter_5000.caffemodel
-rw-r--r-- 1 root root   31468 Aug 17 10:04 custom_net_iter_5000.solverstate
-rw-r--r-- 1 root root    1738 Aug 17 05:23 lenet.prototxt
-rw-r--r-- 1 root root     777 Aug 17 05:23 lenet_adadelta_solver.prototxt
-rw-r--r-- 1 root root     778 Aug 17 05:23 lenet_auto_solver.prototxt
-rw-r--r-- 1 root root    1407 Aug 17 09:14 lenet_auto_test.prototxt
-rw-r--r-- 1 root root    1407 Aug 17 09:14 lenet_auto_train.protot

In [23]:
! cat mnist/custom_auto_solver.prototxt

train_net: "mnist/custom_auto_train.prototxt"
test_net: "mnist/custom_auto_test.prototxt"
test_iter: 100
test_interval: 500
base_lr: 0.01
display: 1000
max_iter: 10000
lr_policy: "inv"
gamma: 0.0001
power: 0.75
momentum: 0.9
weight_decay: 0.0005
snapshot: 5000
snapshot_prefix: "mnist/custom_net"
solver_mode: CPU
random_seed: 831486
type: "SGD"
